In [ ]:
!pip install transformers
!pip install datasets evaluate accelerate optuna optimum
!pip install --upgrade-strategy eager install optimum[onnxruntime]
!pip install onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
import time
import shutil
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import Features, Value, ClassLabel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoConfig
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [ ]:
# dataset=load_dataset("jeswinLLM/VERY_RANDOM_DATA")

In [ ]:
# df = pd.DataFrame(dataset['train'])

In [ ]:
# df.head()

In [ ]:
# class_names = df["label_name"].unique().tolist()

In [ ]:
# class_names

In [ ]:
df = pd.read_csv("dataset.tsv", delimiter="\\t")

<ipython-input-8-e54127428a0e>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("dataset.tsv", delimiter="\\t")


In [ ]:
df.head()

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern


In [ ]:
df.columns

Index(['page_id', 'text', 'label', 'Pattern Category'], dtype='object')

In [ ]:
df = df.drop(columns = ['page_id', 'Pattern Category'],axis=1)

In [ ]:
df.head()

,text,label
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1
1,Pillowcases & Shams,0
2,Write a review,0
3,"To start your return, simply click on the foll...",0
4,newsletter signup (privacy policy),0


In [ ]:
df["label"].value_counts()

0               1178
1               1175
Social Proof       1
Urgency            1
Misdirection       1
Name: label, dtype: int64

In [ ]:
df["label"].unique()

array(['1', '0', None, 'Social Proof', 'Urgency', 'Misdirection'],
      dtype=object)

In [ ]:
remove = [ None, 'Social Proof', 'Urgency', 'Misdirection']

In [ ]:
df = df[~df['label'].isin(remove)]

In [ ]:
df["label"] = df["label"].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_intent, test_intent = train_test_split(df,test_size=0.1, stratify = df["label"])

In [ ]:
train_intent["label"].value_counts(normalize=True)

0    0.500709
1    0.499291
Name: label, dtype: float64

In [ ]:
test_intent["label"].value_counts(normalize=True)

1    0.5
0    0.5
Name: label, dtype: float64

In [ ]:
# train_intent=train_intent.drop(["label_name"],axis=1)
# test_intent=test_intent.drop(["label_name"],axis=1)

In [ ]:
train_intent.to_csv("train_help.csv" , index=False)
test_intent.to_csv("val_help.csv",index=False)


In [ ]:
class_names =  ["Not_Dark_Pattern","Dark_Pattern"]

In [ ]:

ft=Features({'text':Value(dtype='string',id=None), 'label': ClassLabel(num_classes=len(class_names), names=class_names)})
dataset=load_dataset('csv',data_files={'train':'train_help.csv','validation':'val_help.csv'},features=ft)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
model_ckpt = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

helpdesk_encoded = dataset.map(tokenize, batched=True, batch_size=None)
helpdesk_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2117 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_labels = len(class_names)
model = (AutoModelForSequenceClassification
      .from_pretrained(model_ckpt, num_labels=num_labels)
      .to(device))

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
config = (AutoConfig.from_pretrained(model_ckpt,
                                  num_labels=len(class_names),
                                  id2label={i: label for i, label in enumerate(class_names)},
                                  label2id={label: i for i, label in enumerate(class_names)}))

model.config = config

In [ ]:

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}


In [ ]:
batch_size = 4
logging_steps = len(helpdesk_encoded["train"]) // batch_size
model_name = "Venkatesh4342/distilroberta-dark-pattern"
training_args = TrainingArguments(
  output_dir=model_name,
  num_train_epochs=6,
  learning_rate=2e-5,
  evaluation_strategy ='steps',
  eval_steps=500,
  per_device_eval_batch_size=batch_size,
  weight_decay=0.01,
  save_steps=2000,
  save_total_limit=3,
  load_best_model_at_end= True,
  logging_steps=logging_steps,
  gradient_checkpointing=True,
  push_to_hub=True)

In [ ]:
trainer = Trainer(model=model,
                args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=helpdesk_encoded["train"],
                eval_dataset=helpdesk_encoded["validation"],
                tokenizer=tokenizer)

trainer.train()

In [ ]:
trainer.push_to_hub("Training complete!")

events.out.tfevents.1702391833.70c1b6a8bd40.467.1:   0%|          | 0.00/8.78k [00:00<?, ?B/s]

'https://huggingface.co/Venkatesh4342/distilroberta-helpdesk-intent/tree/main/'

In [ ]:
trainer.save_model("fine_tuned_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1704461652.9a4afcd28b25.249.0:   0%|          | 0.00/6.36k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:


onnx_model = ORTModelForSequenceClassification.from_pretrained("fine_tuned_model", export=True)
quantizer = ORTQuantizer.from_pretrained(onnx_model)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
model_quantized_path = quantizer.quantize(
  save_dir="model",
  quantization_config=dqconfig,
)
time.sleep(15)


In [ ]:
api.upload_folder(
  folder_path="model",
  repo_id="Venkatesh4342/quantized-dark_pattern",
  repo_type="model"
)

In [ ]:
quant_sent_model = ORTModelForSequenceClassification.from_pretrained("Venkatesh4342/quantized-dark_pattern")
quant_sent_tokenizer = AutoTokenizer.from_pretrained("Venkatesh4342/quantized-dark_pattern")

config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

model_quantized.onnx:   0%|          | 0.00/82.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

In [ ]:
pipe =  pipeline("text-classification", model=quant_sent_model, tokenizer=quant_sent_tokenizer)

In [ ]:
result = pipe("win exciting prize of $10000000")

In [ ]:
result

[{'label': 'Dark_Pattern', 'score': 0.9999727010726929}]

In [ ]:
from typing import List, Optional, Union

In [ ]:
import transformers

In [ ]:
class dark_pattern_recognition:
  def __init__(
        self,
        intent_pipe,
    ):
        self.intent_pipeline = intent_pipe

  @classmethod
  def load_model(
        cls,
        dark_pattern_model: Optional[str] = "Venkatesh4342/quantized-dark_pattern",
     ):

        quant_intent_model = ORTModelForSequenceClassification.from_pretrained(dark_pattern_model)
        quant_intent_tokenizer = AutoTokenizer.from_pretrained(dark_pattern_model)
        intent_pipe =  transformers.pipeline("text-classification", model=quant_intent_model, tokenizer=quant_intent_tokenizer )

        return cls(intent_pipe)

  def __call__(
        self,
        input: Optional[str],
    ):
      intent = self.intent_pipeline(input)

      return intent[0]["label"]

In [ ]:
pipe = dark_pattern_recognition.load_model()

In [ ]:
res = pipe("Refund is available here")

In [ ]:
res

'Not_Dark_Pattern'

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=1c891974c164e93c2268495da4cc707fe744621230aa838b8a80d22d815e8292
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_OU5ZLOBMMAUDV5FB5DGIZW6H-W6LOYY6OAZSPQWSG")

In [ ]:
@anvil.server.callable
def predict_dark_pattern(statement0):
  out0 = pipe(statement0)
  return out0

In [ ]:
anvil.server.wait_forever()